In [0]:
%sql
create or replace table pubmed_db.coautor_2023
select distinct pmid, substr(year, 1, 4) AS year, authors from pubmed_temp_filtrado
where  substr(year,1,4) = 2023 and authors.orcid is not null


num_affected_rows,num_inserted_rows


## CREAR ARISTAS Y NODOS

In [0]:


from pyspark.sql.functions import explode, col, regexp_replace, count, coalesce

# 1. Leer tabla con autores
df_pubmed = spark.table("pubmed_db.coautor_2023")

# 2. Explode autores, filtrar ORCID válido
df_authors = df_pubmed.select("pmid", explode("authors").alias("author")) \
    .select(
        "pmid",
        regexp_replace(col("author.orcid"), "https://orcid.org/", "").alias("orcid"),
        col("author.fullname").alias("author_name")
    ).filter(col("orcid").rlike("^[0-9]{4}-[0-9]{4}-[0-9]{4}-[0-9]{4}$"))

# 3. Crear aristas de coautoría (pares de ORCID por artículo)
df_edges = df_authors.alias("a1").join(df_authors.alias("a2"), on="pmid") \
    .filter(col("a1.orcid") < col("a2.orcid")) \
    .select(col("a1.orcid").alias("src"), col("a2.orcid").alias("dst"))

# 4. Calcular peso (número de colaboraciones)
df_weighted_edges = df_edges.groupBy("src", "dst").count().withColumnRenamed("count", "weight")

# 5. Calcular grado para cada autor
df_degree_src = df_weighted_edges.groupBy("src").agg(count("*").alias("degree_src"))
df_degree_dst = df_weighted_edges.groupBy("dst").agg(count("*").alias("degree_dst"))

df_degree = df_degree_src.join(df_degree_dst, df_degree_src.src == df_degree_dst.dst, "full_outer") \
    .selectExpr(
        "coalesce(src, dst) as orcid",
        "coalesce(degree_src, 0) + coalesce(degree_dst, 0) as degree"
    )

# 6. Unir nombres y grado
df_authors_unique = df_authors.select("orcid", "author_name").distinct()

df_nodes = df_authors_unique.join(df_degree, on="orcid", how="left").fillna(0)

# 7. Guardar tablas en Delta (reemplaza rutas según tu workspace)
df_nodes.write.format("delta").mode("overwrite").saveAsTable("pubmed_coauthor_nodes_2023")
df_weighted_edges.write.format("delta").mode("overwrite").saveAsTable("pubmed_coauthor_edges_2023")

# 8. Mostrar muestra pequeña para visualización (100 nodos + aristas relacionadas)
sample_nodes = df_nodes.limit(100).select("orcid").collect()
sample_ids = [row["orcid"] for row in sample_nodes]

sample_edges = df_weighted_edges \
    .filter((col("src").isin(sample_ids)) & (col("dst").isin(sample_ids)))

display(df_nodes.filter(col("orcid").isin(sample_ids)))
display(sample_edges)

orcid,author_name,degree
0000-0002-3245-3296,Michael Gray,8
0000-0002-0310-6842,Haijun Jiang,4
0000-0002-0228-6204,Giuseppe Chiazzese,8
0000-0002-6853-9884,Omar S Akbari,25
0000-0002-6853-9884,Omar Akbari,25
0000-0001-9663-3088,Zhiping Deng,5
0000-0002-3414-5481,Anna-Clara Rullander,1
0000-0002-1679-9561,Vitalii Kavun,5
0000-0001-6253-1286,T P Oscar,0
0000-0001-7192-7821,Markus Drechsler,12


src,dst,weight
0000-0003-4829-5955,0009-0001-1758-9910,1
0000-0002-9715-8634,0009-0000-0738-7744,1
0000-0003-4481-1289,0009-0003-3405-0372,1
0009-0007-9850-3376,0009-0008-3663-8854,2
0000-0001-5278-2794,0000-0001-8607-3195,1
0000-0003-1459-4527,0000-0003-4396-9623,1


## Crear **visualizaacion**

In [0]:


# ------------------------------
# 0) Librerías
# ------------------------------
import networkx as nx
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from pyspark.sql.functions import col

# ------------------------------
# 1) Pasa muestra a Pandas
# ------------------------------
# Filtra solo los nodos en la muestra (por ejemplo ORCID)
nodes_pd = df_nodes.filter(col("orcid").isin(sample_ids)).toPandas()

# Filtra TOP N coautorías (mayores pesos)
TOP_N = 20
edges_pd = sample_edges.orderBy(col("weight").desc()).limit(TOP_N).toPandas()

# ------------------------------
# 2) Normalizar degree → tamaño y color de nodos
# ------------------------------
max_degree = nodes_pd["degree"].max()
nodes_pd["viz_size"] = nodes_pd["degree"] / max_degree * 30  # Tamaño máx ajusta a gusto (p.ej. 30)

# Paleta viridis para nodos
norm_degree = mcolors.Normalize(vmin=0, vmax=max_degree)
cmap_nodes = cm.get_cmap('viridis')

def degree_to_color(degree):
    rgba = cmap_nodes(norm_degree(degree))
    r, g, b, a = [int(255*x) if i < 3 else x for i, x in enumerate(rgba)]
    return {'r': r, 'g': g, 'b': b, 'a': a}

nodes_pd["viz_color"] = nodes_pd["degree"].apply(degree_to_color)

# ------------------------------
# 3) Normalizar weight → grosor y color de aristas
# ------------------------------
max_weight = edges_pd["weight"].max()
edges_pd["viz_thickness"] = edges_pd["weight"] / max_weight * 10  # Grosor máx 10

# Paleta plasma para aristas
norm_weight = mcolors.Normalize(vmin=0, vmax=max_weight)
cmap_edges = cm.get_cmap('plasma')

def weight_to_color(weight):
    rgba = cmap_edges(norm_weight(weight))
    r, g, b, a = [int(255*x) if i < 3 else x for i, x in enumerate(rgba)]
    return {'r': r, 'g': g, 'b': b, 'a': a}

edges_pd["viz_color"] = edges_pd["weight"].apply(weight_to_color)

# ------------------------------
# 4) Crear grafo con viz
# ------------------------------
G = nx.Graph()

# Añadir nodos con label y tamaño
for idx, row in nodes_pd.iterrows():
    G.add_node(
        row["orcid"],
        label=row["author_name"],
        degree=row["degree"],
        viz={'size': row["viz_size"], 'color': row["viz_color"]}
    )

# Añadir solo aristas TOP N
for idx, row in edges_pd.iterrows():
    G.add_edge(
        row["src"],
        row["dst"],
        weight=row["weight"],
        viz={'thickness': row["viz_thickness"], 'color': row["viz_color"]}
    )

# ------------------------------
# 5) Guardar GEXF listo para Gephi
# ------------------------------
nx.write_gexf(G, "/dbfs/FileStore/pubmed/sample_graph_topN_viz_2023.gexf")

print("✅ GEXF exportado: /dbfs/FileStore/pubmed/sample_graph_topN_viz_2023.gexf")


## https://3132215626649366.6.gcp.databricks.com/files/pubmed/sample_graph_topN_viz_2023.gexf

/root/.ipykernel/1606/command-4367526046110856-1061313050:28: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap_nodes = cm.get_cmap('viridis')
/root/.ipykernel/1606/command-4367526046110856-1061313050:45: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap_edges = cm.get_cmap('plasma')


✅ GEXF exportado: /dbfs/FileStore/pubmed/sample_graph_topN_viz_2023.gexf
